# 在mindspore框架下实现 LeNet-5 在MNIST数据集上的训练和测试，实验代码如下

In [ ]:
import os
import mindspore as ms
import mindspore.dataset as ds
import mindspore.dataset.vision.transforms as vision
from mindspore import context, nn, Model
from mindspore.nn.metrics import Accuracy
from mindspore.common.initializer import TruncatedNormal
from mindspore.nn.optim.momentum import Momentum
import mindspore.dataset.transforms as transforms
from mindspore.dataset import vision



# 由于下载的mindspore版本是cpu版本，所以将训练设备设置为cpu

In [ ]:
# 设置设备为CPU
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

# 数据集路径
mnist_dataset_dir = r"E:\Pycharm\pythonProject\Le-Net-5\mindspore\MNIST\raw"

# 定义数据集的文件目录结构


label_trans = transforms.TypeCast(ms.int32)
data_trans = ms.dataset.transforms.Compose([transforms.TypeCast(ms.float32), vision.HWC2CHW()])



# 使用官方提供的数据集进行定义数据集 并且定义Le-Net-5模型

In [ ]:
# 定义数据集
def create_dataset(train=True):
    if train:
        return ds.MnistDataset(dataset_dir=mnist_dataset_dir, usage='train', shuffle=True).\
            map(operations=data_trans).batch(batch_size=4).map(operations=label_trans, input_columns=["label"])
    else:
        return ds.MnistDataset(dataset_dir=mnist_dataset_dir, usage='test', shuffle=True).\
            map(operations=data_trans).batch(batch_size=4).map(operations=label_trans, input_columns=["label"])

# 定义LeNet-5模型
class MyLeNet5(nn.Cell):
    def __init__(self, num_class=10, num_channel=1):
        super(MyLeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=num_channel, out_channels=6, kernel_size=5, padding=0, stride=1)
        self.conv2 = nn.Conv2d(6, 16, 5, padding=0, stride=1)

        self.fc1 = nn.Dense(784, 120, weight_init=TruncatedNormal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=TruncatedNormal(0.02))
        self.fc3 = nn.Dense(84, num_class, weight_init=TruncatedNormal(0.02))

        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def construct(self, x):
        x = self.max_pool2d(self.relu(self.conv1(x)))
        x = self.max_pool2d(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 创建LeNet-5模型实例
net = MyLeNet5()

# 定义损失函数和优化器
loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.9)



#  创建模型、训练模型、测试模型

In [ ]:

model = Model(net, loss_fn, optimizer, metrics={'accuracy': Accuracy()})


model.train(5, create_dataset(train=True))


result = model.eval(create_dataset(train=False))
print("Test Accuracy:", result['accuracy'])
